In [1]:
import pandas as pd
import networkx as nx
import matplotlib

%matplotlib qt

In [153]:
df = pd.read_csv('data/proc_MA_1000_albums_non_cumulative.csv')
df.dropna(axis=0)#.head()


,artist,album,ignored tags,image,listeners,mbid,playcount,release-date,tags,url,MA_score,MA_artist,MA_album
5,Mayhem,Deathcrush,[],[{'#text': 'https://lastfm.freetls.fastly.net/...,97480.0,08f88d68-c0bf-474a-8707-43cd58e28ab6,1557715.0,1987-03,['black metal'],https://www.last.fm/music/Mayhem/Deathcrush,23.46,Mayhem,Deathcrush
8,Celtic Frost,Monotheist,[],[{'#text': 'https://lastfm.freetls.fastly.net/...,96350.0,af74f70f-4542-447e-a32c-4a8c9e69e6bd,1978261.0,2006,"['doom metal', 'black metal', 'thrash metal', ...",https://www.last.fm/music/Celtic+Frost/Monotheist,21.45,Celtic Frost,Monotheist
10,Blind Guardian,A Night at the Opera,[],[{'#text': 'https://lastfm.freetls.fastly.net/...,95668.0,fb4a053e-7139-49e4-9aa6-bf1f29299b3c,2729237.0,2002-03-05,"['power metal', 'symphonic metal', 'progressiv...",https://www.last.fm/music/Blind+Guardian/A+Nig...,20.28,Blind Guardian,A Night at the Opera
17,Metallica,Master of Puppets,[],[{'#text': 'https://lastfm.freetls.fastly.net/...,949182.0,b30c90dd-83f1-4897-ae36-58fb3935e5ee,22464934.0,2003,"['thrash metal', 'heavy metal']",https://www.last.fm/music/Metallica/Master+of+...,31.83,Metallica,Master of Puppets
19,Immortal,At the Heart of Winter,[],[{'#text': 'https://lastfm.freetls.fastly.net/...,94190.0,8c8395a0-3a02-35d6-a975-535112ec057e,1911912.0,1999-02-28,['black metal'],https://www.last.fm/music/Immortal/At+the+Hear...,25.09,Immortal,At the Heart of Winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...
844,Behemoth,The Satanist,[],[{'#text': 'https://lastfm.freetls.fastly.net/...,117783.0,8f2a9448-cec0-41aa-b518-3e067c65a336,4875419.0,2014-02-03,"['blackened death metal', 'black metal', 'deat...",https://www.last.fm/music/Behemoth/The+Satanist,24.84,Behemoth,The Satanist
848,Death,Leprosy,[],[{'#text': 'https://lastfm.freetls.fastly.net/...,115042.0,146f04e6-4995-3047-932c-9029767f7385,2484094.0,1988-11-16,['death metal'],https://www.last.fm/music/Death/Leprosy,20.57,Death,Leprosy
850,Sepultura,Beneath the Remains,[],[{'#text': 'https://lastfm.freetls.fastly.net/...,114792.0,e15200cf-7606-4ce8-b6b1-8e73b7161b88,2468535.0,1989-11-05,"['thrash metal', 'death metal']",https://www.last.fm/music/Sepultura/Beneath+th...,27.08,Sepultura,Beneath the Remains
859,Burzum,Hvis lyset tar oss,[],[{'#text': 'https://lastfm.freetls.fastly.net/...,112403.0,b6928219-0772-39ac-8156-91a609b2fd5e,1594703.0,1994-04,"['black metal', 'ambient black metal', 'atmosp...",https://www.last.fm/music/Burzum/Hvis+lyset+ta...,28.39,Burzum,Hvis lyset tar oss


In [3]:
ignored_tags = []

In [154]:
df['ignored tags'].dropna(axis=0)

0      []
1      []
2      []
3      []
4      []
       ..
901    []
902    []
903    []
904    []
905    []
Name: ignored tags, Length: 906, dtype: object

In [155]:
tags_df = df[['tags','ignored tags']].dropna(axis=0)

In [156]:
ign_tags = tags_df['ignored tags'].values
tags = tags_df['tags'].values

In [157]:
ign_tags_list = []

for ign_tag in ign_tags:
    ign_tags_list.append(eval(ign_tag))

tags_list = []

for tag in tags:
    tags_list.append(eval(tag))

In [158]:
import itertools
ign_tags_merged = list(set(list(itertools.chain.from_iterable(ign_tags_list))))
tags_merged = list(set(list(itertools.chain.from_iterable(tags_list))))

In [159]:
ign_tags_merged

[]

In [160]:
tags_merged[:10]

['goregrind',
 'gothic metal',
 'grunge',
 'deathcore',
 'death metal',
 'extreme metal',
 'symphonic metal',
 'avant-garde black metal',
 'blackgaze',
 'pirate metal']

In [162]:
G = nx.Graph()
G.add_nodes_from(tags_merged) #create a node for each tag
nx.set_node_attributes(G, 1,'weight')


In [163]:
for d in tags_list:
    # Increase node weight
    for n in d:
        G.nodes[n]['weight'] += 1

    # Draw Edges (or increase weight if already present)
    if len(d) >= 2:
        for comb in itertools.combinations(d, 2):
            u = comb[0]
            v = comb[1]
            if G.has_edge(u, v):
                G[u][v]['weight'] += 1
            else:
                G.add_edge(u, v, weight=1)


In [151]:
n_weights = nx.get_node_attributes(G, 'weight')
edges,e_weights = zip(*nx.get_edge_attributes(G,'weight').items())

plt.rcParams['axes.facecolor'] = 'white'
# COLOR = 'white'
# matplotlib.rcParams['text.color'] = COLOR

plt.figure(1,figsize=(12,12)) 
pos = nx.circular_layout(G)
nx.draw_networkx(G, pos, nodelist=n_weights.keys(), node_size=[v * 10 for v in n_weights.values()], edgelist=edges, edge_color=e_weights, width=1.0, edge_cmap=plt.cm.gist_yarg)
plt.show()